<a href="https://colab.research.google.com/github/albertotrunk/depth2video/blob/main/02_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy

In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
).to("cuda")

In [ ]:
import requests
from PIL import Image
from IPython.display import display
import numpy as np
from torchvision.transforms import GaussianBlur

url_mask = "https://raw.githubusercontent.com/albertotrunk/depth2video/main/example/depth.png?raw=true" #@param {type:"string"}
mask_image = Image.open(requests.get(url_mask, stream=True).raw).convert('RGB')


mask_image = mask_image.convert('L')

mask = np.array(mask_image)

strength = 1  # this controls the "strength" of the prompt relative to the init image

d = int(255 * (1-strength))
mask *= 255-d # convert from range [0,1] to [0,255]
mask += d

mask_image = Image.fromarray(mask)

display(mask_image)

# fuzzing the mask edges generally improves synthesis results

blur = GaussianBlur(11,20)
mask_image = blur(mask_image)


url_init = "https://raw.githubusercontent.com/albertotrunk/depth2video/main/example/color.png?raw=true" #@param {type:"string"}
init_image = Image.open(requests.get(url_init, stream=True).raw)
display(init_image)


In [ ]:
#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
prompt = "a photo of Scarlett Johansson with a short haircut wearing a white sweater, hyper resolution 10k" #@param {type:"string"}
negative_prompt = "border, frame, lowres, jpeg artifacts, low quality, normal quality, worst quality, poorly drawn, error. abstract, asymmetrical, blurry, cropped, disconnected, duplicate, extra, missing, signature, text, username, watermark. amputee, bad anatomy, deformed, disfigured, disproportionate, eye bags, fused, malformed, morbid, mutated, mutation, mutilated, hands, fingers,bad hand,extra limbs,missing limb,extra leg,missing leg,missing hand,extra hand"  #@param {type:"string"}

guidance_scale = 5  #@param {type:"number"}

image = pipe(prompt=prompt, negative_prompt=negative_prompt, guidance_scale=guidance_scale, image=init_image, mask_image=mask_image).images[0]
image

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = [init_image,mask_image,image]


plt.figure(figsize=(20,10))
columns = 3
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(image)